<p style = 'color:#E6E4E7 ; background-color:#00561B ; text-align:center ; font-size: 300%'> Kayak project - Get hotel data for Bayeux

<p style = 'color:#00561B ; background-color:#E6E4E7 ; text-align:center ; font-size: 150%'> Project prepared by Stephanie Cotineau - #dsmpt-Paris-08

In [1]:
!pip install Scrapy -q

In [2]:
import os # For manipulate file 
import logging # For display logs
import scrapy # Import scrapy & scrapy.crawler
from scrapy.crawler import CrawlerProcess
import re

In [3]:
# Name of the file where the results will be saved
filename = "Bayeux_multipage.json"

# If file already exists, delete it before crawling (because Scrapy will 
# concatenate the last and new results otherwise)
if filename in os.listdir('results_hotels/'):
        os.remove('results_hotels/' + filename)

# Declare a new CrawlerProcess with some settings
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36'
process = CrawlerProcess(settings = {
    'USER_AGENT': user_agent,
    'LOG_LEVEL': logging.ERROR,
    "FEEDS": {
        'results_hotels/' + filename : {"format": "json"},
    }
})

In [4]:
from bs4 import BeautifulSoup 
import requests

In [5]:
# Bayeux - 5 first pages
offset = 0
link_offset = []
while offset != 125:
    url = f"https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=Bayeux%2C+Basse-Normandie%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=bayeux&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1410836&dest_type=city&place_id_lat=49.27605&place_id_lon=-0.70229&search_pageview_id=15c43bf6873c02b4&search_selected=true&offset={offset}"
    #print(url)
    offset = offset + 25
    link_offset.append(url)

In [6]:
link_offset

['https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=Bayeux%2C+Basse-Normandie%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=bayeux&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1410836&dest_type=city&place_id_lat=49.27605&place_id_lon=-0.70229&search_pageview_id=15c43bf6873c02b4&search_selected=true&offset=0',
 'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46Ad

In [7]:
class BayeuxSpider(scrapy.Spider) :
    # Name of spider
    name = 'Bayeux'
    
    # Starting URL
    start_urls = ['https://www.booking.com/']
    
    def start_requests(self):
        for d in ['https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=Bayeux%2C+Basse-Normandie%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=bayeux&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1410836&dest_type=city&place_id_lat=49.27605&place_id_lon=-0.70229&search_pageview_id=15c43bf6873c02b4&search_selected=true&offset=0',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=Bayeux%2C+Basse-Normandie%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=bayeux&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1410836&dest_type=city&place_id_lat=49.27605&place_id_lon=-0.70229&search_pageview_id=15c43bf6873c02b4&search_selected=true&offset=25',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=Bayeux%2C+Basse-Normandie%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=bayeux&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1410836&dest_type=city&place_id_lat=49.27605&place_id_lon=-0.70229&search_pageview_id=15c43bf6873c02b4&search_selected=true&offset=50',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=Bayeux%2C+Basse-Normandie%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=bayeux&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1410836&dest_type=city&place_id_lat=49.27605&place_id_lon=-0.70229&search_pageview_id=15c43bf6873c02b4&search_selected=true&offset=75',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB&sid=318bebf61bd36e4cdf8b55a0d0f1f93c&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AuyYwpQGwAIB0gIkODExM2E0ZjMtNzgzZS00NDg2LTllZTItOTQ3MjQwZjhjZDhj2AIE4AIB%26sid%3D318bebf61bd36e4cdf8b55a0d0f1f93c%26sb_price_type%3Dtotal%26%26&ss=Bayeux%2C+Basse-Normandie%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=bayeux&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1410836&dest_type=city&place_id_lat=49.27605&place_id_lon=-0.70229&search_pageview_id=15c43bf6873c02b4&search_selected=true&offset=100']:
            yield scrapy.Request(d,callback=self.search)
    
    # Callback used after search launched
    def search(self, response):
        hotels = response.css('div.a826ba81c4.fe821aea6c.fa2f36ad22.afd256fc79.d08f526e0d.ed11e24d01.da89aeb942')
        for hotel in hotels:
            yield {
                'location' : hotel.css('span.f4bd0794db.b4273d69aa::text').get(),
                'hotel_url' : hotel.css('a.e13098a59f').attrib['href'],
                'hotel_name' : hotel.css('div.fcab3ed991.a23c043802::text').get(),
                'hotel_rating' : hotel.css('div.b5cd09854e.d10a6220b4::text').get(),
                'hotel_description_short' : hotel.css('div.d8eab2cf7f::text').get()
            }

In [8]:
# Start the crawling using the spider defined above
process.crawl(BayeuxSpider)
process.start()

In [9]:
import pandas as pd

In [10]:
bayeux = pd.read_json('results_hotels/Bayeux_multipage.json')

In [11]:
len(bayeux)

125

In [12]:
bayeux.head(55)

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short
0,Bayeux,https://www.booking.com/hotel/fr/centre-histor...,FREEDOM APPART - CENTRE HISTORIQUE - LUMINEUX ...,"9,1",Hébergement géré par un particulier
1,Bayeux,https://www.booking.com/hotel/fr/saint-patrice...,Hôtel Le Saint Patrice,"8,4",L’Hôtel Le Saint Patrice vous accueille à Baye...
2,Bayeux,https://www.booking.com/hotel/fr/le-manoir-des...,Le Manoir des Doyens,"8,9",Hébergement géré par un particulier
3,Bayeux,https://www.booking.com/hotel/fr/novotel.fr.ht...,Novotel Bayeux,"7,8",Le Novotel Bayeux est situé entre Caen et Cher...
4,Bayeux,https://www.booking.com/hotel/fr/hotelduluxemb...,Grand Hôtel du Luxembourg et La Table du Grand...,"7,8",Occupant une demeure de caractère du XVIIe siè...
5,Bayeux,https://www.booking.com/hotel/fr/particulier-q...,"Hôtel particulier ""le clos de la croix""","9,8","Offrant une vue sur son jardin, l'Hôtel partic..."
6,Bayeux,https://www.booking.com/hotel/fr/appartement-l...,appartement l'étoile,"9,6","Doté d'un court de tennis, l'appartement l'éto..."
7,Bayeux,https://www.booking.com/hotel/fr/la-closerie-t...,La Closerie Teranga,"8,9",Hébergement géré par un particulier
8,Bayeux,https://www.booking.com/hotel/fr/la-naomath.fr...,"La Naomath - Maison d'hôtes, Hébergement insol...","8,3","Situé à seulement 6 km du centre de Bayeux, la..."
9,Bayeux,https://www.booking.com/hotel/fr/villa-camelia...,Villa Camélia,"8,8",Hébergement géré par un particulier


In [13]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.0) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/13.0.782.112 Safari/535.1'}
result_ll =[]
for link in bayeux['hotel_url']:
    page = requests.get(link, headers = headers)
    soup_link = BeautifulSoup(page.content, "lxml")
    lat_lon = soup_link.find_all(id = "hotel_address")
    for ll in lat_lon : 
        #print(ll['data-atlas-latlng'])
        result_ll.append(ll['data-atlas-latlng'])

In [14]:
len(result_ll)

125

In [15]:
bayeux['Latitude_Longitude'] = result_ll

In [16]:
lat = []
lon = []

# For each row in a varible,
for row in bayeux['Latitude_Longitude']:
    # Try to,
    try:
        # Split the row by comma and append
        # everything before the comma to lat
        lat.append(row.split(',')[0])
        # Split the row by comma and append
        # everything after the comma to lon
        lon.append(row.split(',')[1])
    # But if you get an error
    except:
        # append a missing value to lat
        lat.append(np.NaN)
        # append a missing value to lon
        lon.append(np.NaN)

# Create two new columns from lat and lon
bayeux['latitude'] = lat
bayeux['longitude'] = lon

In [17]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.0) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/13.0.782.112 Safari/535.1'}
result_long_description =[]
for link in bayeux['hotel_url']:
    page = requests.get(link, headers = headers)
    soup_link = BeautifulSoup(page.content, "lxml")
    long_description = soup_link.find_all(id = "property_description_content") 
    for ld in long_description : 
        result_long_description.append(ld.text)

In [18]:
len(result_long_description)

125

In [19]:
bayeux['hotel_description_long'] = result_long_description

In [20]:
bayeux.head(55)

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short,Latitude_Longitude,latitude,longitude,hotel_description_long
0,Bayeux,https://www.booking.com/hotel/fr/centre-histor...,FREEDOM APPART - CENTRE HISTORIQUE - LUMINEUX ...,"9,1",Hébergement géré par un particulier,"49.27814786,-0.70626530",49.27814786,-0.70626530,\nVous pouvez bénéficier d'une réduction Geniu...
1,Bayeux,https://www.booking.com/hotel/fr/saint-patrice...,Hôtel Le Saint Patrice,"8,4",L’Hôtel Le Saint Patrice vous accueille à Baye...,"49.27981023,-0.71067492",49.27981023,-0.71067492,\nL’Hôtel Le Saint Patrice vous accueille à Ba...
2,Bayeux,https://www.booking.com/hotel/fr/le-manoir-des...,Le Manoir des Doyens,"8,9",Hébergement géré par un particulier,"49.26616863,-0.71424276",49.26616863,-0.71424276,"\nSitué à Saint-Loup-Hors, à 20 minutes à pied..."
3,Bayeux,https://www.booking.com/hotel/fr/novotel.fr.ht...,Novotel Bayeux,"7,8",Le Novotel Bayeux est situé entre Caen et Cher...,"49.28176922,-0.71848869",49.28176922,-0.71848869,\nLe Novotel Bayeux est situé entre Caen et Ch...
4,Bayeux,https://www.booking.com/hotel/fr/hotelduluxemb...,Grand Hôtel du Luxembourg et La Table du Grand...,"7,8",Occupant une demeure de caractère du XVIIe siè...,"49.27842191,-0.70329666",49.27842191,-0.70329666,\nVous pouvez bénéficier d'une réduction Geniu...
5,Bayeux,https://www.booking.com/hotel/fr/particulier-q...,"Hôtel particulier ""le clos de la croix""","9,8","Offrant une vue sur son jardin, l'Hôtel partic...","49.27796400,-0.70167700",49.27796400,-0.70167700,"\nOffrant une vue sur son jardin, l'Hôtel part..."
6,Bayeux,https://www.booking.com/hotel/fr/appartement-l...,appartement l'étoile,"9,6","Doté d'un court de tennis, l'appartement l'éto...","49.27893640,-0.70770880",49.27893640,-0.70770880,\nVous pouvez bénéficier d'une réduction Geniu...
7,Bayeux,https://www.booking.com/hotel/fr/la-closerie-t...,La Closerie Teranga,"8,9",Hébergement géré par un particulier,"49.27804172,-0.70187662",49.27804172,-0.70187662,\nVous pouvez bénéficier d'une réduction Geniu...
8,Bayeux,https://www.booking.com/hotel/fr/la-naomath.fr...,"La Naomath - Maison d'hôtes, Hébergement insol...","8,3","Situé à seulement 6 km du centre de Bayeux, la...","49.24770809,-0.65629363",49.24770809,-0.65629363,\nVous pouvez bénéficier d'une réduction Geniu...
9,Bayeux,https://www.booking.com/hotel/fr/villa-camelia...,Villa Camélia,"8,8",Hébergement géré par un particulier,"49.27992250,-0.71092490",49.27992250,-0.71092490,\nVous pouvez bénéficier d'une réduction Geniu...


In [21]:
bayeux['hotel_description_long'] = bayeux['hotel_description_long'].replace('\n','', regex=True)

In [22]:
bayeux.head(5)

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short,Latitude_Longitude,latitude,longitude,hotel_description_long
0,Bayeux,https://www.booking.com/hotel/fr/centre-histor...,FREEDOM APPART - CENTRE HISTORIQUE - LUMINEUX ...,"9,1",Hébergement géré par un particulier,"49.27814786,-0.70626530",49.27814786,-0.70626530,Vous pouvez bénéficier d'une réduction Genius ...
1,Bayeux,https://www.booking.com/hotel/fr/saint-patrice...,Hôtel Le Saint Patrice,"8,4",L’Hôtel Le Saint Patrice vous accueille à Baye...,"49.27981023,-0.71067492",49.27981023,-0.71067492,L’Hôtel Le Saint Patrice vous accueille à Baye...
2,Bayeux,https://www.booking.com/hotel/fr/le-manoir-des...,Le Manoir des Doyens,"8,9",Hébergement géré par un particulier,"49.26616863,-0.71424276",49.26616863,-0.71424276,"Situé à Saint-Loup-Hors, à 20 minutes à pied d..."
3,Bayeux,https://www.booking.com/hotel/fr/novotel.fr.ht...,Novotel Bayeux,"7,8",Le Novotel Bayeux est situé entre Caen et Cher...,"49.28176922,-0.71848869",49.28176922,-0.71848869,Le Novotel Bayeux est situé entre Caen et Cher...
4,Bayeux,https://www.booking.com/hotel/fr/hotelduluxemb...,Grand Hôtel du Luxembourg et La Table du Grand...,"7,8",Occupant une demeure de caractère du XVIIe siè...,"49.27842191,-0.70329666",49.27842191,-0.70329666,Vous pouvez bénéficier d'une réduction Genius ...


In [23]:
bayeux['hotel_description_long'][55]

"Offrant une vue sur le jardin, le DOMAINE DU GRAND CAUGY est situé à Bayeux, à 2,3 km du musée de la Tapisserie de Bayeux. Vous bénéficierez gratuitement d'une connexion Wi-Fi et d'un parking privé.Les logements comprennent une télévision à écran plat ainsi qu'une salle de bains privative pourvue d'une douche et d'un sèche-cheveux. Leur kitchenette est équipée d'un micro-ondes et d'un réfrigérateur.Ce Bed & Breakfast sert un petit-déjeuner continental tous les matins.Vous pourrez également vous détendre dans le jardin.Vous séjournerez à 2,9 km du musée Baron Gérard et à 3 km de la cathédrale Notre-Dame de Bayeux. L'aéroport de Caen-Carpiquet, le plus proche, est implanté à 31 km. "

In [24]:
# Clean location column
bayeux['location'] = ["Bayeux" if value!="Bayeux" else value for value in bayeux["location"]]

In [25]:
bayeux.head()

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short,Latitude_Longitude,latitude,longitude,hotel_description_long
0,Bayeux,https://www.booking.com/hotel/fr/centre-histor...,FREEDOM APPART - CENTRE HISTORIQUE - LUMINEUX ...,"9,1",Hébergement géré par un particulier,"49.27814786,-0.70626530",49.27814786,-0.70626530,Vous pouvez bénéficier d'une réduction Genius ...
1,Bayeux,https://www.booking.com/hotel/fr/saint-patrice...,Hôtel Le Saint Patrice,"8,4",L’Hôtel Le Saint Patrice vous accueille à Baye...,"49.27981023,-0.71067492",49.27981023,-0.71067492,L’Hôtel Le Saint Patrice vous accueille à Baye...
2,Bayeux,https://www.booking.com/hotel/fr/le-manoir-des...,Le Manoir des Doyens,"8,9",Hébergement géré par un particulier,"49.26616863,-0.71424276",49.26616863,-0.71424276,"Situé à Saint-Loup-Hors, à 20 minutes à pied d..."
3,Bayeux,https://www.booking.com/hotel/fr/novotel.fr.ht...,Novotel Bayeux,"7,8",Le Novotel Bayeux est situé entre Caen et Cher...,"49.28176922,-0.71848869",49.28176922,-0.71848869,Le Novotel Bayeux est situé entre Caen et Cher...
4,Bayeux,https://www.booking.com/hotel/fr/hotelduluxemb...,Grand Hôtel du Luxembourg et La Table du Grand...,"7,8",Occupant une demeure de caractère du XVIIe siè...,"49.27842191,-0.70329666",49.27842191,-0.70329666,Vous pouvez bénéficier d'une réduction Genius ...


In [26]:
bayeux.to_csv("Bayeux.csv")